# GRAIN 2:
Modelos de clasificación de trigo y maiz, se entrena un modelo para clasificar todo el grano.

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

import  imblearn
from imblearn.over_sampling import  ADASYN


import matplotlib.pyplot as plt
from tqdm import tqdm
import datetime
#from PIL import image
#import tensorflow-gpu
#from bayes_opt import BayesianOptimization


print(tf.__version__)
#tf.debugging.set_log_device_placement(True)
#-------------------------------------------------------------------------------
import time




Iniciamos configurando tensorflow para usar la GPU, al igual que tensorboard

In [ ]:
"""log_dir = "logs/both/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)"""

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if len(tf.config.list_physical_devices('GPU'))>0:
    physical_devices = tf.config.list_physical_devices('GPU')
    print(physical_devices)
    tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    print("Virtual GPU devices: ", tf.config.list_logical_devices('GPU'))

In [ ]:
tf.debugging.set_log_device_placement(False)

## Ambos granos: Trigo y Maiz
Generamos dos iteradores de carpetas para los conjuntos de datos de validación y de entrenamiento.

In [ ]:
# Directorios de entrenamiento y validación
train_dir = '../grainspace/pre/both/train'
validation_dir = '../grainspace/pre/both/validation'

datagen = ImageDataGenerator(
    rescale=1./255,          # Normaliza los valores de los píxeles
    shear_range=0.2,         # Aplica transformaciones de corte
    zoom_range=0.2,          # Aplica zoom aleatorio
    horizontal_flip=True     # Invierte las imágenes horizontalmente
)
train_generator = datagen.flow_from_directory(
    train_dir,  # Directorio con las imágenes de entrenamiento
    target_size=(150, 150),     # Tamaño de las imágenes
    batch_size=32,              # Tamaño del lote
    class_mode='categorical'         # Tipo de clasificación (binary, categorical, etc.)
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)






In [ ]:
batch_images, batch_labels = next(train_generator)

# Assuming the images are in the first channel of the batch
sample_images = batch_images[:, :, :, 0]

# Plot the first few images
num_images_to_plot = 5
fig, axes = plt.subplots(1, num_images_to_plot, figsize=(15, 5))
for i in range(num_images_to_plot):
    axes[i].imshow(sample_images[i])
    axes[i].axis('on')
plt.show()

Definimos la arquitectura del modelo, una red neural convolucional de 4 capas convolucionales, y dos capas densas.

In [ ]:
print(train_generator.class_indices)

In [ ]:
dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=(150, 150, 3)))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
            
            model.add(Dropout(0.5))
            model.add(Dense(len(train_generator.class_indices)))
            model.add(Activation('softmax'))
            
            

            tensorboard = TensorBoard(log_dir="logs/G600_both/{}".format(NAME))
            
            model.summary()
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

            history = model.fit(
                train_generator,
                steps_per_epoch=train_generator.samples // train_generator.batch_size,
                validation_data=validation_generator,
                validation_steps=validation_generator.samples // validation_generator.batch_size,
                epochs=30,
                callbacks=[tensorboard, early_stopping],
                batch_size=32
                )
            # Evaluación en conjunto de validación
            loss, accuracy = model.evaluate(validation_generator)
            print(f'Loss: {loss}')
            print(f'Accuracy: {accuracy}')
            

In [ ]:
"""model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(train_generator.class_indices), activation='softmax')

])"""

In [ ]:
"""model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),  # Capa de convolución adicional
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),  # Capa densa adicional
    keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])"""